#TP2 :   CALCULER LA MATRICE STATIONNAIRE  P*

  **Réalisé par:**  ROM OUMAYMA                                            
              **Encadrer par :** Prof.Driss Ait Omar

##ETAPE  1 : Trouver les classes récurrentes et les états transitoires

In [ ]:
# Fonction pour determiner les etats qui peuvent etre atteints depuis un etat donner dans une chaine M
def reachable_states(P, state):
    n = P.shape[0]
    reachable = set([state])
    stack = [state]

    while stack:
        current = stack.pop()
        for next_state in range(n):
            if P[current, next_state] > 0 and next_state not in reachable:
                reachable.add(next_state)
                stack.append(next_state)

    return reachable

# Fonction pour trouver les classes recurrentes
def communication_class(P, state):
    reachable_from_state = reachable_states(P, state)
    reachable_to_state = reachable_states(P.T, state)
    return reachable_from_state.intersection(reachable_to_state)

def find_recurrent_classes(P):
    n = P.shape[0]
    visited = set()
    recurrent_classes = []

    for i in range(n):
        if i not in visited:
            current_class = communication_class(P, i)
            is_recurrent = True
            for state in current_class:
                for next_state in range(n):
                    if P[state, next_state] > 0 and next_state not in current_class:
                        is_recurrent = False
                        break
                if not is_recurrent:
                    break
            if is_recurrent:
                recurrent_classes.append(current_class)
            visited.update(current_class)

    return recurrent_classes


# Fonction pour trouver les etats transitoires
def find_transient_states(P):
    recurrent_classes = find_recurrent_classes(P)

    recurrent_states = set()
    for recurrent_class in recurrent_classes:
        recurrent_states.update(recurrent_class)

    transient_states = {i for i in range(P.shape[0]) if i not in recurrent_states}

    return transient_states

##ETAPE 2 : Resolution du systeme stationnaire pour chaque classe recurrente Ek

In [ ]:


def solve_stationary_for_class(P, recurrent_class):
    n = len(recurrent_class)
    A = np.eye(n) - P[np.ix_(recurrent_class, recurrent_class)].T
    A[-1, :] = np.ones(n)  # Contrainte de somme = 1
    b = np.zeros(n)
    b[-1] = 1
    pi_class = np.linalg.solve(A, b)

    return pi_class

# Fonction pour calculer la matrice stationnaire
def calculate_stationary_matrix(P):
    recurrent_classes = find_recurrent_classes(P)
    transient_states = find_transient_states(P)

    n = P.shape[0]
    stationary_matrix = np.zeros((n, n))

    for rec_class in recurrent_classes:
        sorted_class = sorted(list(rec_class))
        pi_class = solve_stationary_for_class(P, sorted_class)
        for state in sorted_class:
            stationary_matrix[state, sorted_class] = pi_class


    # Gestion des états transitoires
    for i in transient_states:
        # Somme des contributions de tous les états accessibles
        stationary_matrix[i, :] = np.sum([P[i, j] * stationary_matrix[j, :] for j in range(n)], axis=0)
        # Normalisation pour éviter la somme nulle
        if np.sum(stationary_matrix[i, :]) > 0:  # Vérification pour éviter la division par zéro
            stationary_matrix[i, :] /= np.sum(stationary_matrix[i, :])

    return stationary_matrix


##ETAPE 3 :Organiser les probabilités stationnaires 𝑃*



In [ ]:
# Fonction pour reorganiser la matrice stationnaire
def reorder_stationary_matrix(P, stationary_matrix):
    recurrent_classes = find_recurrent_classes(P)
    transient_states = find_transient_states(P)

    new_order = []
    for rec_class in recurrent_classes:
        new_order.extend(sorted(list(rec_class)))
    new_order.extend(sorted(list(transient_states)))

    reordered_matrix = stationary_matrix[np.ix_(new_order, new_order)]

    return reordered_matrix, new_order

#EXEMPLE


In [ ]:
import numpy as np

# Exemple de matrice de transition P
P = np.array([[0.5, 0.5, 0, 0, 0, 0],
              [2/3, 1/3, 0, 0, 0, 0],
              [0, 0, 0.5, 0.5, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0]])


# Appel des fonctions
recurrent_classes = find_recurrent_classes(P)
transient_states = find_transient_states(P)

print(f"Classes récurrentes : {[{s + 1 for s in cl} for cl in recurrent_classes]}")
print(f"États transitoires : {[s + 1 for s in transient_states]}")

# Calcul de la matrice stationnaire
stationary_matrix = calculate_stationary_matrix(P)

# Rorganiser la matrice
reordered_matrix, new_order = reorder_stationary_matrix(P, stationary_matrix)

# Affichage
print("Matrice stationnaire après ordre des états :", [i + 1 for i in new_order])
print(np.array2string(reordered_matrix, separator=', ', formatter={'float_kind':lambda x: f'{x:.2f}'}))

Classes récurrentes : [{1, 2}, {4, 5}]
États transitoires : [3, 6]
Matrice stationnaire après ordre des états : [1, 2, 4, 5, 3, 6]
[[0.57, 0.43, 0.00, 0.00, 0.00, 0.00],
 [0.57, 0.43, 0.00, 0.00, 0.00, 0.00],
 [0.00, 0.00, 0.50, 0.50, 0.00, 0.00],
 [0.00, 0.00, 0.50, 0.50, 0.00, 0.00],
 [0.00, 0.00, 0.50, 0.50, 0.00, 0.00],
 [0.00, 0.00, 0.50, 0.50, 0.00, 0.00]]
